In [33]:
import pandas as pd
import numpy as np
from PredictNan import PredictNan
import datetime
import math

In [34]:
train = pd.read_csv("../data/raw/d_train.csv")
test = pd.read_csv("../data/raw/d_test_A.csv")

test_id = test.pop("id")
train_id =  train.pop("id")

total_ID = pd.concat([train_id, test_id])

target = pd.DataFrame(train.pop("血糖"))

total = pd.concat([train,test])

columns = ["*r-谷氨酰基转换酶",
           "*球蛋白",
           "低密度脂蛋白胆固醇",
           "乙肝e抗体",
           "红细胞计数",
           "中性粒细胞%",
           ]
for column in columns:
    total[column] = total[column].apply(lambda x : np.log1p(x))


columns = [
          "红细胞平均血红蛋白量",
          "血小板比积"]
for column in columns:
    total.pop(column)

#添加统计量
columns = ["age","甘油三酯","红细胞平均体积","尿素","尿酸","*碱性磷酸酶","红细胞平均血红蛋白浓度","*天门冬氨酸氨基转换酶",
          "*r-谷氨酰基转换酶",
          ]
for column in columns:
    if column != "date" and column != "gender":
        max_value = total[column].max()
        min_value = total[column].min()
        avg_value = total[column].mean()
        total[column+"max"] = total[column].apply(lambda x : x - max_value)
date = total.pop("date")
one_hot = pd.get_dummies(total["gender"], prefix="gender")
total.pop("gender")
total = pd.concat([one_hot,total],axis=1)

#处理缺失值
columns = total.columns.tolist()
for column in columns:
    #0.908
    #total[column] = total[column].fillna(total[column].mean())
    #0.8000
    total[column] = total[column].fillna(total[column].median())
train = total[0:5642]
train = pd.concat([train_id,train,target],axis=1)

test = total[5642:]
test = pd.concat([test_id,test],axis=1)
train.to_csv("../data/processed/train_best.csv",index=False,encoding="utf8")
test.to_csv("../data/processed/test_best.csv",index=False,encoding="utf8")



In [20]:
train = pd.read_csv("../data/raw/d_train.csv")
test = pd.read_csv("../data/raw/d_test_A.csv")

test_id = test.pop("id")
train_id =  train.pop("id")

total_ID = pd.concat([train_id, test_id])

target = pd.DataFrame(train.pop("血糖"))

total = pd.concat([train,test])

date = total.pop("date")

#处理缺失值
columns = total.columns.tolist()
for column in columns:
    #0.908
    #total[column] = total[column].fillna(total[column].mean())
    #0.8000
    total[column] = total[column].fillna(total[column].median())
    
columns = ["*r-谷氨酰基转换酶",
           "*球蛋白",
           "低密度脂蛋白胆固醇",
           "乙肝e抗体",
           "红细胞计数",
           "中性粒细胞%",
           ]
for column in columns:
    total[column] = total[column].apply(lambda x: np.log1p(x))

In [21]:
columns = ["age", "甘油三酯", "红细胞平均体积", "尿素", "尿酸", "*碱性磷酸酶", "红细胞平均血红蛋白浓度", "*天门冬氨酸氨基转换酶",
           "*r-谷氨酰基转换酶"]
for column in columns:
    if column != "date" and column != "gender":
        max_value = total[column].max()
        min_value = total[column].min()
        avg_value = total[column].mean()
        std_value = total[column].std()

        total[column+"max"] = total[column].apply(lambda x : x - max_value)
        total[column+"min"] = total[column].apply(lambda x : x - min_value)
        total[column+"avg"] = total[column].apply(lambda x : x - avg_value)
        total[column+"std"] = total[column].apply(lambda x : (x - avg_value)/std_value)
columns = ["甘油三酯","尿酸"]
source = "age"
for column in columns:
    total[source+"*"+column] = total.apply(lambda x : x[source] * x[column], axis=1)
    
columns = ["尿酸"]
source = "尿素"
for column in columns:
    total[source+"*"+column] = total.apply(lambda x : x[source] * x[column], axis=1)
    
columns = ["红细胞平均血红蛋白浓度"]
source = "尿酸"
for column in columns:
    total[source+"*"+column] = total.apply(lambda x : x[source] * x[column], axis=1)

drop_columns = ["尿素std"]
for column in drop_columns:
    total.pop(column)


In [22]:
one_hot = pd.get_dummies(total["gender"], prefix="gender")
total.pop("gender")
total = pd.concat([one_hot,total],axis=1)

In [23]:
train = total[0:5642]
train = pd.concat([train_id,train,target],axis=1)

test = total[5642:]
test = pd.concat([test_id,test],axis=1)
train.to_csv("../data/processed/train.csv",index=False,encoding="utf8")
test.to_csv("../data/processed/test.csv",index=False,encoding="utf8")